In [3]:
import MySQLdb
import requests
import urllib.parse
import time
from bs4 import BeautifulSoup
from selenium import webdriver

conn = MySQLdb.connect(host="localhost", user="root", passwd="", db="python",charset='utf8')#連結資料庫
cur = conn.cursor()

Google_url='https://www.google.com/'
mobile01_url='https://www.mobile01.com/'
sqli = "insert into mobile01 (key_word,search_title,search_href,search_time,search_author,article_picture,author_href,search_view) values (%s,%s,%s,%s,%s,%s,%s,%s)" #選擇資料表

#driver=webdriver.Chrome(executable_path=r'E://chromedriver') #模擬瀏覽器
driver=webdriver.PhantomJS(executable_path=r'E://phantomjs/bin/phantomjs') #模擬瀏覽器
driver.maximize_window()

key_word='吐司'
payload={'q':key_word,'site':'site:https://www.mobile01.com/'}
q1=urllib.parse.quote(payload['q']) #url編碼轉換
url='https://google.com.tw/search?q='+q1+'+'+payload['site']
driver.get(url)

print(url)

page=1
for change in range(2,4):
    pageSource = driver.page_source #重新讀取當前頁面
    soup = BeautifulSoup (pageSource, "lxml") #流入BeautifulSoup

    for link in soup.select('.r'):
        
        #ChromeDriver
        #search_href=link.find('a')['href']
        
        
        #PhantomJS
        search_href=Google_url + link.find('a')['href']
        print(search_href)
        
        res=requests.get(search_href)
        res.encoding='utf-8'
        soup2 = BeautifulSoup (res.text, "lxml")
        href_split=search_href.split('%')
        if href_split[0]=='https://www.google.com//url?q=https://www.mobile01.com/topicdetail.php'or href_split[0]=='https://www.google.com//url?q=https://www.mobile01.com/waypointtopicdetail.php':
            Main_article = soup2.find_all('main')
            #標題
            search_title=Main_article[0].select('.topic')[0].text
            print(search_title)
            #作者、作者網址
            authors = Main_article[0].select('.inner')
            author_name = authors[0].select('.fn')
            search_author=author_name[0].find('a').text
            author_href=mobile01_url+author_name[0].find('a')['href']

            #人氣
            info=authors[0].select('.info')
            search_view=info[0].text.strip('文章人氣:').lstrip().replace(',','')
            #文章時間
            time1=authors[0].select('.date')
            search_time=time1[0].text.strip('#1')
            #文章圖片
            post = Main_article[0].select('.single-post-content')
            if post[0].find('img')!=None:
                picture=post[0].find('img')['src'].strip()
                split=picture.split('/')
                if split[2]=='attach.mobile01.com':
                    article_picture='https:'+picture
                elif split[2]=='download.mobile01.com':
                    article_picture='https:'+picture
                elif split[2]=='attach2.mobile01.com':
                    article_picture='./img/mobile01.jpg'
                else:
                    article_picture=picture
            else:
                article_picture='./img/mobile01.jpg'
            cur.execute(sqli,(key_word,search_title,search_href,search_time,search_author,article_picture,author_href,search_view)) #存入資料庫    
            conn.commit()
        time.sleep(0.5)
                    
    

    a=repr(change)
    driver.find_element_by_link_text(a).click() #點擊下一頁
    print('此處頁數為第',page,'頁')
    page+=1
    print ("===============================================================================================================")

driver.close();
cur.close() #斷開連結
conn.close()


https://google.com.tw/search?q=%E5%90%90%E5%8F%B8+site:https://www.mobile01.com/
https://www.google.com//url?q=https://www.mobile01.com/topicdetail.php%3Ff%3D602%26t%3D3678764&sa=U&ved=0ahUKEwjJtIvimt7TAhVMGJQKHXQ5AdMQFggTMAA&usg=AFQjCNGaiBI3DvzlsOY2o98XDkKwxECbaw
32兩吐司內縮問題
https://www.google.com//url?q=https://www.mobile01.com/topicdetail.php%3Ff%3D602%26t%3D3266928&sa=U&ved=0ahUKEwjJtIvimt7TAhVMGJQKHXQ5AdMQFggZMAE&usg=AFQjCNEMrUUzCvp7oQB1DcfRv81L0MlqvA
請教：有關12兩吐司模，12兩的定義？
https://www.google.com//url?q=https://www.mobile01.com/topicdetail.php%3Ff%3D602%26t%3D3329080&sa=U&ved=0ahUKEwjJtIvimt7TAhVMGJQKHXQ5AdMQFggfMAI&usg=AFQjCNGIRBlU9uaF_fNyPcSDEpctOnIDCQ
吐司出爐成品有洞.是哪步驟出了問題
https://www.google.com//url?q=https://www.mobile01.com/topicdetail.php%3Ff%3D602%26t%3D4819056&sa=U&ved=0ahUKEwjJtIvimt7TAhVMGJQKHXQ5AdMQFgglMAM&usg=AFQjCNHav2Umax1UYd8kMVojLLHZtBt6-w
請問麵包機做的吐司怎麼樣才會好吃？
https://www.google.com//url?q=https://www.mobile01.com/topicdetail.php%3Ff%3D602%26t%3D2725919&sa=U&ved=0ahUKEwjJtIvi